# #of users added/updated

Can pull this from pagelog dump or activity file

Both files are huge

Would probably lean toward activity file so we don't have to recreate filtering logic

biq_query_activity.parquet in \\sandhills.int\systems\devFramework\Machine_Learning\spark_data\recsys_files\daily_refresh

New user if min(click_datetime) is less than a day old

Updated user otherwise

Alternative would be Neo4j (probably preferred)

In [61]:
print('hello')

hello


# #of listings added/updated

Use the csv files in \\sandhills.int\systems\devFramework\Machine_Learning\spark_data\dbd_files\live\active_listings

Filter to listings created on or modified on within the last day

Alternative would be Neo4j (still probably lean active listings files though)

In [62]:
import pandas as pd
import datetime as dt
import pytz
import os
def get_new_stuff(path, filename):
    file_path = os.path.join(path, filename)
    filter_date = (dt.datetime.now(pytz.timezone('UTC')) - dt.timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    item_df = pd.read_csv(file_path, iterator=True, chunksize=1000)
    df = pd.concat([chunk[(chunk['ListingCreatedOnUTCDateTime'] > filter_date)|(chunk['ListingModifiedOnUTCDateTime']>filter_date)] for chunk in item_df])
    # item_df['ListingCreatedOnUTCDateTime'] = pd.to_datetime(item_df['ListingCreatedOnUTCDateTime'])
    # item_df['ListingModifiedOnUTCDateTime'] = pd.to_datetime(item_df['ListingModifiedOnUTCDateTime'])
    # item_df = item_df[(item_df['ListingCreatedOnUTCDateTime'] > filter_date) | (item_df['ListingModifiedOnUTCDateTime'] > filter_date)]
    return df.shape[0]
env = 'live'
active_listing_path = f'/mnt/Machine_Learning/spark_data/dbd_files/{env}/active_listings'
retail_name = 'ActiveRetailListing.csv'
auction_name = 'ATEFActiveListings.csv'
retail = get_new_stuff(active_listing_path,retail_name)
auction = get_new_stuff(active_listing_path, auction_name)





# #of active listings in Qdrant

Use qdrant client to count points in the user collection

https://qdrant.tech/documentation/concepts/points/#counting-points 

http://siml4:9217/dashboard#/collections/live_recsys_activity

http://siml4:9217/dashboard#/collections/live_recsys_similar_items

Can pull from active listings CSV dump, but prefer Qdrant here because some listings get filtered out due to API failures

In [63]:
from pydantic import ValidationError
from qdrant_client import QdrantClient, models

class LoaderBase():
    def __init__(self) -> None:
        super().__init__()
        self.lnk_hostname = 'siml4'  # SIML4
        self.saw_hostname = 'wiml4'

class QdrantLoad(LoaderBase):
    def __init__(self, collection, all_check=True):
        super().__init__()
        self.port = 9217
        self.clients = [
            QdrantClient(self.lnk_hostname, port=self.port, timeout=100000),
            QdrantClient(self.saw_hostname, port=self.port, timeout=100000)
        ]
        self.collection = collection
        self.all_check = all_check
    
    def get_active_listings(self):
        counts = []
        for client in self.clients:
            try:
                count = client.count(
                    collection_name=self.collection,
                    exact=True,
                )
                counts.append(count)
            except ValidationError as e:
                print(f"Validation error fetching collection from {client}: {e}")
            except Exception as e:
                print(f"Error fetching collection from {client}: {e}")
        return counts


env = 'dev'
qdrant_activity = 'recsys_activity'
qdrant_similar_items = 'recsys_similar_items'
qdrant_activity = f'{env}_{qdrant_activity}'
qdrant_similar_items = f'{env}_{qdrant_similar_items}'

qdrant_users = 'recsys_user'
qdrant_users = f'{env}_{qdrant_users}'

QdrantLoad(qdrant_activity).get_active_listings()
QdrantLoad(qdrant_similar_items).get_active_listings()
QdrantLoad(qdrant_users).get_active_listings()
















INFO:httpx:HTTP Request: POST http://siml4:9217/collections/dev_recsys_activity/points/count "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://wiml4:9217/collections/dev_recsys_activity/points/count "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://siml4:9217/collections/dev_recsys_similar_items/points/count "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://wiml4:9217/collections/dev_recsys_similar_items/points/count "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://siml4:9217/collections/dev_recsys_user/points/count "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://wiml4:9217/collections/dev_recsys_user/points/count "HTTP/1.1 200 OK"


[CountResult(count=2385), CountResult(count=2385)]

# #of users from bid prediction
Have to pull that from Redis

In [64]:
import redis
import pandas as pd
class LoaderBase():
    def __init__(self) -> None:
        super().__init__()
        self.lnk_hostname = 'siml4'  # SIML4
        self.saw_hostname = 'wiml4'
class RedisLoad(LoaderBase):
    def __init__(self, key_prefix):
        super().__init__()
        self.port = 9216
        self.db = 0
        self.key_prefix = key_prefix
        self.conns = [
            redis.Redis(host=self.lnk_hostname, port=self.port, db=self.db),
            redis.Redis(host=self.saw_hostname, port=self.port, db=self.db)
        ]
    def get_count_pattern(self):
        count1 = 0
        for conn in self.conns:
            count = 0
            for key in conn.scan_iter(f"{self.key_prefix}*"):
                count+=1
            count1+=count
        return count1
    
bid_pred_prefix = 'dev:bid'
RedisLoad(bid_pred_prefix).get_count_pattern()



neo4j num of users added/updated and num of listings added/updated



In [ ]:

from py_framework.bis.connectors import Neo4jConnector # type: ignore
from py_framework.common.sandhills_config import Config


lnk_connection = Config().get('con_neo4j_LNK', config_section='default')
saw_connection = Config().get('con_neo4j_SAW', config_section='default')
query = """
MATCH (p:User)-[r:CLICKED_ITEM]->(i:Item)
WHERE r.CreatedAt
LIMIT 50
"""
with Neo4jConnector([lnk_connection, saw_connection]) as neo4j:
    ballin1 = neo4j.query(query)[0]
    print(ballin1)
    ballin = neo4j.structure_results(neo4j.query(query)[0])
    print(ballin)

[<Record r.CreatedAt=neo4j.time.DateTime(2024, 7, 9, 20, 21, 1, 540000000, tzinfo=<UTC>) i.Id=231094033>, <Record r.CreatedAt=neo4j.time.DateTime(2024, 7, 9, 20, 21, 48, 560000000, tzinfo=<UTC>) i.Id=231094033>, <Record r.CreatedAt=neo4j.time.DateTime(2024, 7, 9, 20, 21, 48, 170000000, tzinfo=<UTC>) i.Id=231094033>, <Record r.CreatedAt=neo4j.time.DateTime(2024, 7, 27, 20, 6, 1, 277000000, tzinfo=<UTC>) i.Id=235879667>, <Record r.CreatedAt=neo4j.time.DateTime(2024, 7, 24, 17, 3, 10, 917000000, tzinfo=<UTC>) i.Id=235274767>, <Record r.CreatedAt=neo4j.time.DateTime(2024, 7, 24, 17, 3, 11, 363000000, tzinfo=<UTC>) i.Id=235274767>, <Record r.CreatedAt=neo4j.time.DateTime(2024, 7, 24, 17, 3, 10, 460000000, tzinfo=<UTC>) i.Id=235274767>, <Record r.CreatedAt=neo4j.time.DateTime(2024, 7, 19, 3, 59, 40, 870000000, tzinfo=<UTC>) i.Id=235274767>, <Record r.CreatedAt=neo4j.time.DateTime(2024, 7, 18, 14, 36, 51, 27000000, tzinfo=<UTC>) i.Id=235274767>, <Record r.CreatedAt=neo4j.time.DateTime(2024, 7

Sending da emails

In [ ]:
import ssl
import smtplib
import requests
import json
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from smtp2go.core import Smtp2goClient
bid_pred_prefix = 'dev:bid'
env = 'dev'
count_of_bid_pred_users = 0
active_listing_path = f'/mnt/Machine_Learning/spark_data/dbd_files/{env}/active_listings'
retail_name = 'ActiveRetailListing.csv'
auction_name = 'ATEFActiveListings.csv'
retail = get_new_stuff(active_listing_path,retail_name)
auction = get_new_stuff(active_listing_path, auction_name)

qdrant_activity = 'recsys_activity'
qdrant_similar_items = 'recsys_similar_items'
qdrant_activity = f'{env}_{qdrant_activity}'
qdrant_similar_items = f'{env}_{qdrant_similar_items}'

qdrant_users = 'recsys_user'
qdrant_users = f'{env}_{qdrant_users}'

activity_counts = QdrantLoad(qdrant_activity).get_active_listings()
similar_items_counts = QdrantLoad(qdrant_similar_items).get_active_listings()
user_counts = QdrantLoad(qdrant_users).get_active_listings()

activity_count = sum(activity_counts)
similar_items_count = sum(similar_items_counts)
user_count = sum(user_counts)

client = Smtp2goClient(api_key='api-F0A1307427B54E37AD279EE0814E9D2C')

html_content = f"""
<html>
<body>
    <h1>Listing Information</h1>
    <table border="1">
        <tr>
            <th>Category</th>
            <th>Count</th>
        </tr>
        <tr>
            <td>Bid Pred Users</td>
            <td>{count_of_bid_pred_users}</td>
        </tr>
        <tr>
            <td>Retail Listings</td>
            <td>{retail}</td>
        </tr>
        <tr>
            <td>Auction Listings</td>
            <td>{auction}</td>
        </tr>
        <tr>
            <td>Activity Count</td>
            <td>{activity_count}</td>
        </tr>
        <tr>
            <td>Similar Items Count</td>
            <td>{similar_items_count}</td>
        </tr>
        <tr>
            <td>User Count</td>
            <td>{user_count}</td>
        </tr>
    </table>
</body>
</html>
"""
payload = {
   'sender': 'shithead-shi-thead@outlook.com',
   'recipients': ['ehof17@gmail.com', 'elijah-hoffman@sandhills.com'],
   'subject': 'Very tough here',
   'text': 'Test Message',
   'html': html_content,
   'custom_headers': {'Your-Custom-Headers': 'Custom Values'}
}

resp = client.send(**payload)
smtp2goPass = '$unnySands-ButDontTellMe'
#Outlook stmp server
port = 587
smtp_server = "smtp.office365.com"
login = "elijah-hoffman@sandhills.com"
password = "$unshineSANDHILLSBoy24"

receiver_email = "elijah-hoffman@sandhills.com"

message = """\
Subject: Hi there!


Some people say life is easy. The problem with that is we live this life not because it is easy but because it is hard.

"""

context = ssl.create_default_context()


msg = MIMEMultipart('mixed')

sender = 'shithead-shi-thead@outlook.com'
recipient = 'elijah-hoffman@sandhills.com'

msg['Subject'] = 'Your Subject'
msg['From'] = sender
msg['To'] = recipient

text_message = MIMEText('It is a text message.', 'plain')
html_message = MIMEText('It is a html message.', 'html')
msg.attach(text_message)
msg.attach(html_message)

mailServer = smtplib.SMTP('mail.smtp2go.com', 2525) #  8025, 587 and 25 can also be used. 
mailServer.ehlo()
mailServer.starttls()
mailServer.ehlo()
mailServer.login(login, smtp2goPass)
mailServer.sendmail(sender, recipient, msg.as_string())
mailServer.close()


NameError: name 'get_new_stuff' is not defined

In [4]:
from fetchers import FileFetch, QdrantFetch, Neo4jFetch, RedisFetch
swag = 'dev:bid'
%timeit RedisFetch(swag).scan_iter_bench()

1min ± 549 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
